In [17]:
import os
import numpy as np
import pandas as pd
import scipy.stats

# F检验用于检查给定的两个样本方差是否一致
def f_test(group1, group2):
  f = np.var(group1, ddof=1)/np.var(group2, ddof=1)
  nun = len(group1)-1
  dun = len(group2)-1
  p_value = 1-scipy.stats.f.cdf(f, nun, dun)
  return f, p_value

# 显著性阈值
pval = 0.05
import os
print( os .getcwd ())
# 获取数据
train_df = pd.read_csv(r"E:\SFYdataJPG\小脑\论文\datacsv\train_dataset111.csv")
test_df = pd.read_csv(r"E:\SFYdataJPG\小脑\论文\datacsv\test_dataset111.csv")
# 计算训练集和测试集的特征均值和标准差，检查是否存在显著性差异
assert train_df.shape[1]==test_df.shape[1], "训练集和测试集的特征数量应保持一致！"
# type(train_df.shape)#元组类型
nFeatures = train_df.shape[1]-2  #16个特征
featureName = train_df.columns[2:]#获取列名

e:\SFYdataJPG\Code\imagescore5\imagescore5\lefthearttable-all


In [ ]:
nFeatures
featureName

Index(['age', 'fetal clinical gestational age',
       'fetal ultrasound gestational age'],
      dtype='object')

In [18]:
from scipy.stats import ttest_ind
pFeatures = dict()
trainFMean = []
trainFStd = []
testFMean = []
testFStd = []
pvalue = []
for i in range(nFeatures):
    trainFeature = train_df.iloc[:,i+2]
    testFeature = test_df.iloc[:,i+2]

    fm = np.mean(trainFeature)
    fv = np.std(trainFeature)
    tm = np.mean(testFeature)
    tv = np.std(testFeature)

    trainFMean.append(fm)
    trainFStd.append(fv)
    testFMean.append(tm)
    testFStd.append(tv)   

    # # 基于F-test结果检查是否存在方差一致性
    # _, ftest_p = f_test(trainFeature,testFeature)
    # _, p = scipy.stats.ttest_ind(trainFeature,testFeature,equal_var=ftest_p>pval)
    # pvalue.append(p)
    # pFeatures[featureName[i]] = p
    # 使用 T-检验结果检查是否存在显著差异
    _, p = ttest_ind(trainFeature, testFeature, equal_var=True)  # 设置 equal_var=False 表示假设方差不相等
    pvalue.append(p)
    pFeatures[featureName[i]] = p

    print("{:s} : {:.2f} +- {:.2f} | {:.2f} +- {:.2f} | {:.4f}".format(featureName[i],fm,fv,tm,tv, p))
print("\n")
print("=============================== 以上为表1数据 ===============================")
print("\n")


age : 30.90 +- 4.23 | 30.54 +- 4.08 | 0.4926
fetal clinical gestational age : 28.05 +- 4.08 | 28.69 +- 3.76 | 0.2125
fetal ultrasound gestational age : 28.13 +- 4.25 | 28.93 +- 3.91 | 0.1396


=============================== 以上为表1数据 ===============================




In [10]:
data = {
    'featureName': featureName[:].tolist(),
    'trainFMean': trainFMean[:],
    'trainFStd': trainFStd[:],
    'testFMean': testFMean[:],
    'testFStd':  testFStd[:],
    'pvalue': pvalue[:]
}
# print("Length of featureName:", len(featureName))
# print("Length of trainFMean:", len(trainFMean))
# print("Length of trainFStd:", len(trainFStd))
# print("Length of testFMean:", len(testFMean))
# print("Length of testFStd:", len(testFStd))
# print("Length of pvalue:", len(pvalue))

# 创建DataFrame
df = pd.DataFrame(data)
# 设置小数位数
df['trainFMean'] = df['trainFMean'].round(2)
df['trainFStd'] = df['trainFStd'].round(2)
df['testFMean'] = df['testFMean'].round(2)
df['testFStd'] = df['testFStd'].round(2)
df['pvalue'] = df['pvalue'].round(4)
# 创建新列，格式为"列一 ± 列二"
df['trainFMen ± trainFStd'] = df['trainFMean'].astype(str) + ' ± ' + df['trainFStd'].astype(str)
df['testFMen ± testFStd'] = df['testFMean'].astype(str) + ' ± ' + df['testFStd'].astype(str)
# 删除旧的两列
df = df.drop(['trainFMean', 'trainFStd'], axis=1)
df = df.drop(['testFMean', 'testFStd'], axis=1)
# 重新排列列的顺序
df = df[['featureName', 'trainFMen ± trainFStd', 'testFMen ± testFStd', 'pvalue']]
# 打印DataFrame
print(df)
df.to_excel(r'E:\SFYdataJPG\小脑\论文\table1.xlsx',index=False)

                        featureName trainFMen ± trainFStd testFMen ± testFStd  \
0                               age          30.85 ± 4.33        30.34 ± 3.95   
1    fetal clinical gestational age          28.12 ± 4.05        28.61 ± 3.97   
2  fetal ultrasound gestational age           28.2 ± 4.23        28.81 ± 4.16   

   pvalue  
0  0.3229  
1  0.3205  
2  0.2269  
